In [1]:
import pandas as pd
import numpy as np
import math
import pandas as pd
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## datos
sheets = ["SE", "CE", "SO", "NE2", "SE2", "SE3", "NE", "NO", "NO2", "NTE", "NTE2", "SO2", "SUR"]
dfs = pd.read_excel("datos_hist_2020_2025.xlsx", sheet_name=sheets)
df_sureste_crudos = dfs["SE"]
df_sureste_crudos

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR
0,NaT,ppm,ppb,ppb,ppb,ppb,NaN,NaN,mmHg,mmh,%,ppb,kW/m2,°C,km/h,Azimutal
1,2025-01-01 00:00:00,NaN,NaN,NaN,NaN,36,104.0,31.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-01 01:00:00,0.18,2.6,7.5,10.1,37,58.0,30.84,725.1,0,66,4.1,0,17.62,10.7,136
3,2025-01-01 02:00:00,0.18,2.8,13.5,16.2,26,79.0,49.92,725.1,0,66,4.3,0,16.58,5.3,255
4,2025-01-01 03:00:00,0.19,3.1,15.1,18.1,22,66.0,45.87,725.1,0,74,4.1,0,15.5,5.3,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4339,2025-06-30 19:00:00,0.57,3.4,5.8,9.2,16,15.0,6.75,722.1,0.01,88,2,0.024,22.75,15.4,106
4340,2025-06-30 20:00:00,0.6,3.3,6.2,9.6,16,19.0,5.19,722.5,0,89,2.1,0.003,22.72,17,101
4341,2025-06-30 21:00:00,0.54,3,5.3,8.3,17,18.0,4.36,722.7,0.16,88,1.9,0,22.6,15.9,108
4342,2025-06-30 22:00:00,0.47,2.6,3.9,6.6,19,15.0,3.91,722.9,0.09,90,2,0,22.21,18.5,102


In [3]:
# CONFIGURACIÓN DE RANGOS

# Rangos del fabricante (errores de medición)
MFG_RANGES = {
    "PM10":  (0, 1000),
    "PM2.5": (0, 1000),
    "O3":    (0, 1000),
    "NO":    (0, 500),
    "NO2":   (0, 500),
    "NOx":   (0, 500),
    "SO2":   (0, 500),
    "CO":    (0, 50),
    "RH":    (0, 100),
    "WS":    (0, 180),
    "TEMP":  (-50, 50),
    "SR":    (0, 1.4),
    "BP":    (49.9, 824),
    "WD":    (0, 360),
    # RAINF: conteo * 0.01; solo validamos que no sea negativo
    "RAINF": (0, None),
}

# Rangos de operación por AÑO (outliers)
OP_RANGES = {
    2020: {"PM10":(0,800),"PM2.5":(0,205.94),"O3":(0,153),"NO":(0,500),"NO2":(0,200),
           "NOx":(0,500),"SO2":(0,200),"CO":(0,20),"RH":(0,100),"WS":(0,75),
           "TEMP":(0,41),"SR":(0,1.0),"BP":(690,750),"WD":(0,360),"RAINF":(0,30)},
    2021: {"PM10":(0,800),"PM2.5":(0,325),"O3":(0,175),"NO":(0,350),"NO2":(0,100),
           "NOx":(0,400),"SO2":(0,300),"CO":(0,10),"RH":(0,100),"WS":(0,40),
           "TEMP":(-6.5,45),"SR":(0,1.0),"BP":(690,740),"WD":(0,360),"RAINF":(0,80)},
    2022: {"PM10":(0,999),"PM2.5":(0,450),"O3":(0,160),"NO":(0,400),"NO2":(0,175),
           "NOx":(0,420),"SO2":(0,200),"CO":(0,10),"RH":(0,100),"WS":(0,35),
           "TEMP":(-5,45),"SR":(0,1.25),"BP":(700,740),"WD":(0,360),"RAINF":(0,25)},
    2023: {"PM10":(0,900),"PM2.5":(0,800),"O3":(0,175),"NO":(0,500),"NO2":(0,175),
           "NOx":(0,500),"SO2":(0,250),"CO":(0,14),"RH":(0,100),"WS":(0,40),
           "TEMP":(0,45),"SR":(0,1.0),"BP":(690,740),"WD":(0,360),"RAINF":(0,70)},
    2024: {"PM10":(0,999),"PM2.5":(0,999),"O3":(0,180),"NO":(0,400),"NO2":(0,130),
           "NOx":(0,500),"SO2":(0,150),"CO":(0,18),"RH":(0,100),"WS":(0,38),
           "TEMP":(-4,45.5),"SR":(0,1.26),"BP":(687.5,740),"WD":(0,360),"RAINF":(0,50)},
    2025: {"PM10":(0,820),"PM2.5":(0,350),"O3":(0,185),"NO":(0,350),"NO2":(0,175),
           "NOx":(0,400),"SO2":(0,405),"CO":(0,10),"RH":(0,100),"WS":(0,40),
           "TEMP":(-4.5,45),"SR":(0,1.2),"BP":(688,740),"WD":(0,360),"RAINF":(0,25)},
}

# Aliases para normalizar nombres de columnas (ajústalos si tu dataset usa otros)
ALIASES = {
    "TOUT": "TEMP",
    "WSR":  "WS",
    "PRS":  "BP",
    "NOX":  "NOx",
    "Rainf": "RAINF",
    "RainF": "RAINF",
}


In [8]:
def _norm_cols(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out.columns = [str(c).strip() for c in out.columns]
    out.rename(columns={k: v for k, v in ALIASES.items() if k in out.columns}, inplace=True)
    return out

def _to_num(df: pd.DataFrame, cols) -> pd.DataFrame:
    out = df.copy()
    out.replace({"NULL": np.nan, "NaN": np.nan, "": np.nan}, inplace=True)
    for c in cols:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

def _year_op_maps(var: str):
    """Devuelve diccionarios {year: min}, {year: max} para una variable."""
    y2min, y2max = {}, {}
    for y, d in OP_RANGES.items():
        if var in d:
            y2min[y] = d[var][0]
            y2max[y] = d[var][1]
    return y2min, y2max

def check_ranges(df: pd.DataFrame,
                 datetime_col: str = "datetime",
                 year_col: str | None = None,
                 zone_col: str | None = None):
    """
    Agrega flags por variable:
        <var>_meas_error  -> fuera de rango del fabricante
        <var>_oper_outlier -> fuera de rango de operación (según año)
    Devuelve: df_flagged, long_issues, summary (conteos por tipo/variable)
    """
    # 1) normalizar columnas básicas
    df = _norm_cols(df)

    # 2) garantizar columna datetime y year
    if datetime_col in df.columns:
        dt = pd.to_datetime(df[datetime_col], errors="coerce", dayfirst=False)
    else:
        # intenta con 'date' si existe
        if "date" in df.columns:
            dt = pd.to_datetime(df["date"], errors="coerce", dayfirst=False)
        else:
            raise ValueError(f"No encuentro columna '{datetime_col}' ni 'date'")

    df = df.copy()
    df["__datetime__"] = dt

    if year_col and year_col in df.columns:
        years = pd.to_numeric(df[year_col], errors="coerce").astype("Int64")
    else:
        years = df["__datetime__"].dt.year.astype("Int64")

    df["__year__"] = years

    # 3) columnas candidatas presentes
    vars_presentes = [v for v in MFG_RANGES.keys() if v in df.columns]
    # convierte a numérico
    df = _to_num(df, vars_presentes)

    # 4) generar flags
    for var in vars_presentes:
        # --- fabricante
        vmin, vmax = MFG_RANGES[var]
        val = df[var]
        if vmax is None:
            meas_err = (val < vmin)
        else:
            meas_err = (val < vmin) | (val > vmax)
        df[f"{var}_meas_error"] = meas_err.fillna(False)

        # --- operación por año
        y2min, y2max = _year_op_maps(var)
        op_min = df["__year__"].map(y2min)
        op_max = df["__year__"].map(y2max)

        oper_out = (val < op_min) | (val > op_max)
        # si no hay rango para ese año, no marcamos
        oper_out = oper_out & op_min.notna() & op_max.notna()
        df[f"{var}_oper_outlier"] = oper_out.fillna(False)

    # 5) long dataframe con issues
    id_cols = ["__datetime__", "__year__"] + ([zone_col] if zone_col and zone_col in df.columns else [])
    long_rows = []
    for var in vars_presentes:
        for kind in ("meas_error", "oper_outlier"):
            flag_col = f"{var}_{kind}"
            mask = df[flag_col]
            if not mask.any():
                continue
            if kind == "meas_error":
                vmin, vmax = MFG_RANGES[var]
            else:
                y2min, y2max = _year_op_maps(var)
                # mapeo por fila
                vmin = df.loc[mask, "__year__"].map({y: m for y, m in y2min.items()})
                vmax = df.loc[mask, "__year__"].map({y: M for y, M in y2max.items()})

            block = pd.concat([
                df.loc[mask, id_cols],
                pd.Series(var, index=df.index[mask], name="variable"),
                pd.Series(kind, index=df.index[mask], name="issue_type"),
                df.loc[mask, var].rename("value"),
            ], axis=1)

            if isinstance(vmin, pd.Series):
                block["min_allowed"] = vmin.values
                block["max_allowed"] = vmax.values
            else:
                block["min_allowed"] = vmin
                block["max_allowed"] = vmax
            long_rows.append(block)

    issues_long = pd.concat(long_rows, ignore_index=True) if long_rows else pd.DataFrame(
        columns=id_cols + ["variable","issue_type","value","min_allowed","max_allowed"]
    )

    # 6) resumen
    if not issues_long.empty:
        summary = (issues_long
                   .groupby(["variable","issue_type"])
                   .size()
                   .reset_index(name="count")
                   .sort_values(["variable","issue_type"]))
    else:
        summary = pd.DataFrame(columns=["variable","issue_type","count"])
        print("There wasnt errors")

    # 7) limpiar columnas auxiliares; deja datetime/year legibles
    df_flagged = df.drop(columns=["__year__"]).rename(columns={"__datetime__": "datetime"})
    return df_flagged, issues_long, summary

In [9]:
results = {}
for name, df in dfs.items():
    flagged, issues, summary = check_ranges(df, datetime_col="date")
    results[name] = {
        "flagged": flagged,
        "issues": issues,
        "summary": summary
    }

# Create a summary table with counts of issues for each sheet
summary_table = pd.DataFrame([
    {
        "Sheet": name,
        "Total Issues": results[name]["issues"].shape[0]
    }
    for name in dfs.keys()
])

summary_table
df_sureste_flagged, df_sureste_issues, df_sureste_summary = check_ranges(df_sureste_crudos, datetime_col="date")
df_sureste_summary

There wasnt errors
There wasnt errors
There wasnt errors
There wasnt errors
There wasnt errors
There wasnt errors
There wasnt errors
There wasnt errors
There wasnt errors
There wasnt errors


,variable,issue_type,count


In [6]:
# Let's verify the function is working by checking the outputs
print("Function returned successfully!")
print(f"df_sureste_flagged shape: {df_sureste_flagged.shape}")
print(f"df_sureste_issues shape: {df_sureste_issues.shape}")
print(f"df_sureste_summary shape: {df_sureste_summary.shape}")

print("\nColumns in flagged dataframe:")
print([col for col in df_sureste_flagged.columns if '_meas_error' in col or '_oper_outlier' in col])

print("\nSample of original data:")
print(df_sureste_crudos.head())

Function returned successfully!
df_sureste_flagged shape: (4344, 45)
df_sureste_issues shape: (0, 7)
df_sureste_summary shape: (0, 3)

Columns in flagged dataframe:
['PM10_meas_error', 'PM10_oper_outlier', 'PM2.5_meas_error', 'PM2.5_oper_outlier', 'O3_meas_error', 'O3_oper_outlier', 'NO_meas_error', 'NO_oper_outlier', 'NO2_meas_error', 'NO2_oper_outlier', 'NOx_meas_error', 'NOx_oper_outlier', 'SO2_meas_error', 'SO2_oper_outlier', 'CO_meas_error', 'CO_oper_outlier', 'RH_meas_error', 'RH_oper_outlier', 'WS_meas_error', 'WS_oper_outlier', 'TEMP_meas_error', 'TEMP_oper_outlier', 'SR_meas_error', 'SR_oper_outlier', 'BP_meas_error', 'BP_oper_outlier', 'RAINF_meas_error', 'RAINF_oper_outlier']

Sample of original data:
                 date    CO   NO   NO2   NOX   O3   PM10  PM2.5    PRS RAINF  \
0                 NaT   ppm  ppb   ppb   ppb  ppb    NaN    NaN   mmHg   mmh   
1 2025-01-01 00:00:00   NaN  NaN   NaN   NaN   36  104.0  31.00    NaN   NaN   
2 2025-01-01 01:00:00  0.18  2.6   7.5